In [1]:
import numpy as np 
import scipy.stats as ss
import matplotlib.pyplot as plt
plt.style.use('seaborn')

In [36]:
def YTM (pv: float, maturity: int, cashflow: float = 100):
    """
    This function calculates the yield-to-maturity (ytm) of the promised cash flow
    :param pv: the promised cash flow of the annuity where the calculated value of this promised cash flow
    :param cashflow: the provided cashflow per each coupon data
    :param maturity: the maturity of the provided fixed income asset in months (e.g. 3yr = 36 months)
    return: The ytm is calculated as a continuously compounding annual rate
    """
    
    ytm = 0.0
    step = 0.01

    # Calculate the ytm to within 0.000001, i.e., 0.0001% or 0.01 bps
    while (step > 0.000001):

        # Keep increasing the trial ytm until the resulting present value (pv0) is too small (less than pv).
        while (True):

            # Calculate the present value with a discount rate of "ytm".
            pv0 = 0
            for m in range(1, maturity+1):
                pv0 += cashflow * np.math.exp(-ytm * m / 12.0)

            # Is ytm now too big? If not, increase it further by the amount "step".
            if (pv0 > pv):
                ytm += step

            # If so, reduce it by "step" and break out of the "while (1)" loop. Reduce the step size for the next iteration.
            else:
                ytm -= step
                step /= 10.0
                break

    #return the yield-to-maturity as a percent.
    return 100.0 * ytm              

In [68]:
credit_rating = {"AAA":0.035, "AA":0.04, "A":0.045, "BBB":0.05, "Junk":0.06}

In [79]:
def cash_waterfall(cashflow:int, ratings:dict, type:str = "yearly"):
    if type == "yearly":
        ideal_cash = (100 * 20 * 12) / 5
        ratings = list(credit_rating.keys())
        cash = 0

        for i in range(5):
            if cashflow > ideal_cash:
                cash = ideal_cash
                cashflow -= ideal_cash
            else:
                cash = cashflow

            statement = "{} CDO recieved \n{}\n".format(ratings[i], round(cash,2))
            print(statement)
    elif type == "monthly":
        ideal_cash = (100 * 20) / 5
        ratings = list(credit_rating.keys())
        cash = 0

        for i in range(5):
            if cashflow > ideal_cash:
                cash = ideal_cash
                cashflow -= ideal_cash
            else:
                cash = cashflow

            statement = "{} CDO recieved \n{}\n".format(ratings[i], round(cash,2))
            print(statement)

In [39]:
rho = 0.1

In [40]:
N = np.random.normal(loc=0, scale=1, size=20)

In [41]:
cov = np.reshape(np.array([1 if i==j else rho for j in range(20) for i in range(20)]),(20,20))

In [42]:
A = N.dot(cov)

In [43]:
U = ss.norm.cdf(A, loc=0, scale=1)

In [44]:
T = -50*np.log(U)

In [45]:
U

array([3.16248933e-04, 2.02315698e-01, 9.84459650e-02, 9.90908092e-03,
       4.96219850e-01, 1.32395932e-01, 5.21769570e-03, 4.57311415e-01,
       1.03106343e-01, 8.66920502e-01, 4.45651588e-02, 4.81552886e-01,
       1.19438777e-02, 6.81413903e-03, 2.52842414e-01, 1.59948913e-01,
       5.14440818e-02, 8.90903335e-01, 3.28710093e-01, 5.70148668e-02])

In [46]:
T

array([402.94904452,  79.89629707, 115.91237302, 230.71518386,
        35.03681025, 101.09791808, 262.78497054,  39.11953431,
       113.59971857,   7.14039997, 155.54014585,  36.53696085,
       221.37682295, 249.43777783,  68.74944277,  91.64504056,
       148.36299256,   5.77596739,  55.62895471, 143.22216124])

In [57]:
epv = 0
for m in range(1,13):
    cash = sum([100 if i > m/12 else 0 for i in T])
    epv += cash * np.math.exp(-0.03*m/12)

In [58]:
epv

23614.03100910171

In [59]:
YTM(epv,360,100)

3.0308000000000006

In [80]:
cash_waterfall(epv, ratings=credit_rating, type='yearly')

AAA CDO recieved 
4800.0

AA CDO recieved 
4800.0

A CDO recieved 
4800.0

BBB CDO recieved 
4800.0

Junk CDO recieved 
4414.03

